<a href="https://colab.research.google.com/github/redrum88/data_science/blob/main/shooting_1982_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gun Violence, USA

## About Dataset
Gun ownership in the United States is the highest in the world, and constitutionally protected by the Second Amendment to the United States Constitution. Firearms are widely used in the United States for self-defence, hunting, and recreational uses, such as target shooting.

https://www.kaggle.com/datasets/nidzsharma/us-mass-shootings-19822023

# Part 1: Gun Violence Data Cleaning

In [208]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [354]:
#@title Import Numpy, pandas, matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

# Ignore FutureWarnings
import warnings
warnings.filterwarnings("ignore")

In [355]:
#@title import dataset
df = pd.read_csv("/content/drive/MyDrive/dataset/shooting-1982-2023.csv", parse_dates=['date'])
df.head()

,case,location,date,summary,fatalities,injured,total_victims,location.1,age_of_shooter,prior_signs_mental_health_issues,...,weapons_obtained_legally,where_obtained,weapon_type,weapon_details,race,gender,latitude,longitude,type,year
0,Nashville religious school shooting,"Nashville, TN",2023-03-27,"Audrey Hale, 28, who was a former student at t...",6,1,6,School,28,-,...,-,-,"semiautomatic rifle, semiautomatic handgun",-,-,"F (identified as transgender, per police)",-,-,Mass,2023
1,Michigan State University shooting,"East Lansing, Michigan",2023-02-13,"Anthony D. McRae, 43, opened fire at Berkey Ha...",3,5,8,School,43,-,...,yes,-,semiautomatic handguns,-,Black,M,-,-,Mass,2023
2,Half Moon Bay spree shooting,"Half Moon Bay, California",2023-01-23,"Chunli Zhao, 67, suspected of carrying out the...",7,1,8,workplace,67,-,...,-,-,semiautomatic handgun,-,Asian,M,-,-,Spree,2023
3,LA dance studio mass shooting,"Monterey Park, California",2023-01-21,"Huu Can Tran, 72, fled the scene in a white va...",11,10,21,Other,72,yes,...,-,-,semiautomatic assault weapon (Details pending),-,Asian,M,-,-,Mass,2023
4,Virginia Walmart shooting,"Chesapeake, Virginia",2022-11-22,"Andre Bing, 31, who worked as a supervisor at ...",6,6,12,Workplace,31,-,...,-,-,semiautomatic handgun,-,Black,M,-,-,Mass,2022


In [356]:
#@title Check what columns we have
df.columns

Index(['case', 'location', 'date', 'summary', 'fatalities', 'injured',
       'total_victims', 'location.1', 'age_of_shooter',
       'prior_signs_mental_health_issues', 'mental_health_details',
       'weapons_obtained_legally', 'where_obtained', 'weapon_type',
       'weapon_details', 'race', 'gender', 'latitude', 'longitude', 'type',
       'year'],
      dtype='object')

In [357]:
len(df)

141

In [358]:
#@title Top 10 Cases by fatalities

grouped_by_fatalities = df.groupby('case')['fatalities'].sum()
top_10_by_fatalities = grouped_by_fatalities.sort_values(ascending=False).head(10)

import plotly.graph_objs as go

fig = go.Figure()
fig.add_trace(go.Bar(
    x=top_10_by_fatalities.values,
    y=top_10_by_fatalities.index,
    orientation='h',
    text=top_10_by_fatalities.values,
    textposition='auto', 
))

fig.update_layout(
    title='Top 10 Cases by fatalities',
    xaxis_title='Case',
    yaxis_title='Fatalities',
    height=600,
    margin=dict(l=0, r=0, t=50, b=0),
    yaxis=dict(autorange="reversed")
)

fig.show()

In [359]:
#@title Create new columns from `df['date']`
#@markdown  year, quarter, half, month_name, day_of_week columns and show df.head()
df['year'] = df['date'].dt.year
df['quarter'] = df['date'].dt.quarter
df['half'] = df['date'].dt.month.apply(lambda x: 1 if x <= 6 else 2)
df['month_name'] = df['date'].dt.month_name()
df['day_of_week'] = df['date'].dt.day_name()
df.head()

,case,location,date,summary,fatalities,injured,total_victims,location.1,age_of_shooter,prior_signs_mental_health_issues,...,race,gender,latitude,longitude,type,year,quarter,half,month_name,day_of_week
0,Nashville religious school shooting,"Nashville, TN",2023-03-27,"Audrey Hale, 28, who was a former student at t...",6,1,6,School,28,-,...,-,"F (identified as transgender, per police)",-,-,Mass,2023,1,1,March,Monday
1,Michigan State University shooting,"East Lansing, Michigan",2023-02-13,"Anthony D. McRae, 43, opened fire at Berkey Ha...",3,5,8,School,43,-,...,Black,M,-,-,Mass,2023,1,1,February,Monday
2,Half Moon Bay spree shooting,"Half Moon Bay, California",2023-01-23,"Chunli Zhao, 67, suspected of carrying out the...",7,1,8,workplace,67,-,...,Asian,M,-,-,Spree,2023,1,1,January,Monday
3,LA dance studio mass shooting,"Monterey Park, California",2023-01-21,"Huu Can Tran, 72, fled the scene in a white va...",11,10,21,Other,72,yes,...,Asian,M,-,-,Mass,2023,1,1,January,Saturday
4,Virginia Walmart shooting,"Chesapeake, Virginia",2022-11-22,"Andre Bing, 31, who worked as a supervisor at ...",6,6,12,Workplace,31,-,...,Black,M,-,-,Mass,2022,4,2,November,Tuesday


In [360]:
#@title Plot Cases by Year, Quarter, Half, Month, Day of Week
import plotly.graph_objs as go

# plot by year
fig_year = go.Figure(go.Bar(
    x=df['year'].value_counts().index,
    y=df['year'].value_counts().values,
    name='By Year'
))

fig_year.update_layout(
    title='Cases by Year',
    xaxis_title='Year',
    yaxis_title='Number of Cases',
    height=400,
    margin=dict(l=0, r=0, t=50, b=0),
)

# plot by quarter
fig_quarter = go.Figure(go.Bar(
    x=df['quarter'].value_counts().index,
    y=df['quarter'].value_counts().values,
    name='By Quarter'
))

fig_quarter.update_layout(
    title='Cases by Quarter',
    xaxis_title='Quarter',
    yaxis_title='Number of Cases',
    height=400,
    margin=dict(l=0, r=0, t=50, b=0),
)

# plot by half of year
fig_half = go.Figure(go.Bar(
    x=df['half'].value_counts().index,
    y=df['half'].value_counts().values,
    name='By Half of Year'
))

fig_half.update_layout(
    title='Cases by Half of Year',
    xaxis_title='Half of Year',
    yaxis_title='Number of Cases',
    height=400,
    margin=dict(l=0, r=0, t=50, b=0),
)

# plot by month name
fig_month = go.Figure(go.Bar(
    x=df['month_name'].value_counts().index,
    y=df['month_name'].value_counts().values,
    name='By Month Name'
))

fig_month.update_layout(
    title='Cases by Month Name',
    xaxis_title='Month Name',
    yaxis_title='Number of Cases',
    height=400,
    margin=dict(l=0, r=0, t=50, b=0),
)

# plot by day of week
fig_day = go.Figure(go.Bar(
    x=df['day_of_week'].value_counts().index,
    y=df['day_of_week'].value_counts().values,
    name='By Day of Week'
))

fig_day.update_layout(
    title='Cases by Day of Week',
    xaxis_title='Day of Week',
    yaxis_title='Number of Cases',
    height=400,
    margin=dict(l=0, r=0, t=50, b=0),
)

fig_year.show()
fig_quarter.show()
fig_half.show()
fig_month.show()
fig_day.show()


In [361]:
#@title Plot Cases by Location
import plotly.express as px

# count the number of cases by location
cases_by_location = df['location.1'].value_counts()

# create a bar chart
fig = px.bar(x=cases_by_location.index, y=cases_by_location.values)

# customize the layout
fig.update_layout(
    title='Cases by Location',
    xaxis_title='Location',
    yaxis_title='Number of Cases',
    height=600,
    margin=dict(l=0, r=0, t=50, b=0),
    xaxis_tickangle=-45,
)

# show the plot
fig.show()

In [362]:
#@title Plot Cases by Type
# count the number of cases by type
cases_by_type = df['type'].value_counts()

# create a bar chart
fig = px.bar(x=cases_by_type.index, y=cases_by_type.values)

# customize the layout
fig.update_layout(
    title='Cases by Type',
    xaxis_title='Type',
    yaxis_title='Number of Cases',
    height=600,
    margin=dict(l=0, r=0, t=50, b=0),
)

# show the plot
fig.show()

In [363]:
#@title Plot Cases by Gender
# count the number of cases by gender
cases_by_gender = df['gender'].value_counts()

# create a bar chart
fig = px.bar(x=cases_by_gender.index, y=cases_by_gender.values)

# customize the layout
fig.update_layout(
    title='Cases by Gender',
    xaxis_title='Gender',
    yaxis_title='Number of Cases',
    height=600,
    margin=dict(l=0, r=0, t=50, b=0),
)

# show the plot
fig.show()

In [364]:
df.head()

,case,location,date,summary,fatalities,injured,total_victims,location.1,age_of_shooter,prior_signs_mental_health_issues,...,race,gender,latitude,longitude,type,year,quarter,half,month_name,day_of_week
0,Nashville religious school shooting,"Nashville, TN",2023-03-27,"Audrey Hale, 28, who was a former student at t...",6,1,6,School,28,-,...,-,"F (identified as transgender, per police)",-,-,Mass,2023,1,1,March,Monday
1,Michigan State University shooting,"East Lansing, Michigan",2023-02-13,"Anthony D. McRae, 43, opened fire at Berkey Ha...",3,5,8,School,43,-,...,Black,M,-,-,Mass,2023,1,1,February,Monday
2,Half Moon Bay spree shooting,"Half Moon Bay, California",2023-01-23,"Chunli Zhao, 67, suspected of carrying out the...",7,1,8,workplace,67,-,...,Asian,M,-,-,Spree,2023,1,1,January,Monday
3,LA dance studio mass shooting,"Monterey Park, California",2023-01-21,"Huu Can Tran, 72, fled the scene in a white va...",11,10,21,Other,72,yes,...,Asian,M,-,-,Mass,2023,1,1,January,Saturday
4,Virginia Walmart shooting,"Chesapeake, Virginia",2022-11-22,"Andre Bing, 31, who worked as a supervisor at ...",6,6,12,Workplace,31,-,...,Black,M,-,-,Mass,2022,4,2,November,Tuesday


In [365]:
df.gender.value_counts()

Male                                         70
M                                            65
Male & Female                                 2
Female                                        2
F (identified as transgender, per police)     1
F                                             1
Name: gender, dtype: int64

In [366]:
# Replace "M" with "Male" and "F" with "Female"
df['gender'] = df['gender'].replace({'M': 'Male', 'F': 'Female', 'F (identified as transgender, per police)': "Female (transgender)"})
df['gender'].value_counts()

Male                    135
Female                    3
Male & Female             2
Female (transgender)      1
Name: gender, dtype: int64

In [367]:
df.head()

,case,location,date,summary,fatalities,injured,total_victims,location.1,age_of_shooter,prior_signs_mental_health_issues,...,race,gender,latitude,longitude,type,year,quarter,half,month_name,day_of_week
0,Nashville religious school shooting,"Nashville, TN",2023-03-27,"Audrey Hale, 28, who was a former student at t...",6,1,6,School,28,-,...,-,Female (transgender),-,-,Mass,2023,1,1,March,Monday
1,Michigan State University shooting,"East Lansing, Michigan",2023-02-13,"Anthony D. McRae, 43, opened fire at Berkey Ha...",3,5,8,School,43,-,...,Black,Male,-,-,Mass,2023,1,1,February,Monday
2,Half Moon Bay spree shooting,"Half Moon Bay, California",2023-01-23,"Chunli Zhao, 67, suspected of carrying out the...",7,1,8,workplace,67,-,...,Asian,Male,-,-,Spree,2023,1,1,January,Monday
3,LA dance studio mass shooting,"Monterey Park, California",2023-01-21,"Huu Can Tran, 72, fled the scene in a white va...",11,10,21,Other,72,yes,...,Asian,Male,-,-,Mass,2023,1,1,January,Saturday
4,Virginia Walmart shooting,"Chesapeake, Virginia",2022-11-22,"Andre Bing, 31, who worked as a supervisor at ...",6,6,12,Workplace,31,-,...,Black,Male,-,-,Mass,2022,4,2,November,Tuesday


In [368]:
df["prior_signs_mental_health_issues"].value_counts()

Yes         58
-           28
Unclear     23
No          17
yes          8
TBD          5
Unknown      1
Unclear      1
Name: prior_signs_mental_health_issues, dtype: int64

In [369]:
df["prior_signs_mental_health_issues"] = df["prior_signs_mental_health_issues"].replace({'-': 'Unclear', 'Unknown': 'Unclear', 'Unclear ': "Unclear", 'yes': 'Yes'})
df["prior_signs_mental_health_issues"].value_counts()

Yes        66
Unclear    53
No         17
TBD         5
Name: prior_signs_mental_health_issues, dtype: int64

In [370]:
df.head().T

,0,1,2,3,4
case,Nashville religious school shooting,Michigan State University shooting,Half Moon Bay spree shooting,LA dance studio mass shooting,Virginia Walmart shooting
location,"Nashville, TN","East Lansing, Michigan","Half Moon Bay, California","Monterey Park, California","Chesapeake, Virginia"
date,2023-03-27 00:00:00,2023-02-13 00:00:00,2023-01-23 00:00:00,2023-01-21 00:00:00,2022-11-22 00:00:00
summary,"Audrey Hale, 28, who was a former student at t...","Anthony D. McRae, 43, opened fire at Berkey Ha...","Chunli Zhao, 67, suspected of carrying out the...","Huu Can Tran, 72, fled the scene in a white va...","Andre Bing, 31, who worked as a supervisor at ..."
fatalities,6,3,7,11,6
injured,1,5,1,10,6
total_victims,6,8,8,21,12
location.1,School,School,workplace,Other,Workplace
age_of_shooter,28,43,67,72,31
prior_signs_mental_health_issues,Unclear,Unclear,Unclear,Yes,Unclear


In [371]:
df["weapons_obtained_legally"].value_counts()

Yes                                                                                                                                    81
-                                                                                                                                      17
No                                                                                                                                     16
yes                                                                                                                                     9
TBD                                                                                                                                     7
Unknown                                                                                                                                 6
\nYes                                                                                                                                   2
Kelley passed federal criminal bac

In [372]:
df["weapons_obtained_legally"] = df["weapons_obtained_legally"].replace({"-": "Unknown", "yes": "Yes", "\nYes": "Yes",
                                                                         "Kelley passed federal criminal background checks; the US Air Force failed to provide information on his criminal history to the FBI": "Yes",
                                                                         'Yes ("some of the weapons were purchased legally and some of them may not have been")': "Yes", "Yes ": "Yes"})
df["weapons_obtained_legally"].value_counts()

Yes        95
Unknown    23
No         16
TBD         7
Name: weapons_obtained_legally, dtype: int64

In [373]:
df["where_obtained"].value_counts()

-                                          46
Unknown                                    15
Unclear                                     4
TBD                                         3
Purchased from an individual                2
                                           ..
Florida Gun Center                          1
Sharpshooters Small Arms Range              1
Local gun store in Killeen, Guns Galore     1
Gun was his father's.                       1
Garcia Gun Center in Hialeah, Fla.          1
Name: where_obtained, Length: 76, dtype: int64

In [374]:
df['where_obtained'] = df['where_obtained'].replace({'-': "Unknown"})
df['where_obtained'].value_counts()

Unknown                                         61
Unclear                                          4
TBD                                              3
Purchased from an individual                     2
Issued by Forest County Sheriff's Department     1
                                                ..
Florida Gun Center                               1
Sharpshooters Small Arms Range                   1
Local gun store in Killeen, Guns Galore          1
Gun was his father's.                            1
Garcia Gun Center in Hialeah, Fla.               1
Name: where_obtained, Length: 75, dtype: int64

In [375]:
df['race'].value_counts()

white              41
White              31
Black              16
-                  13
Latino             11
Asian              10
black               9
Other               5
Native American     3
White               1
unclear             1
Name: race, dtype: int64

In [376]:
df['race'] = df['race'].replace({
    'white': 'White',
    'black': 'Black',
    'White ': 'White',
    '-': 'Unclear',
    'unclear': 'Unclear'
})
df['race'].value_counts()

White              73
Black              25
Unclear            14
Latino             11
Asian              10
Other               5
Native American     3
Name: race, dtype: int64

In [377]:
df["location.1"].value_counts()

Other          50
Workplace      45
School         22
Religious       7
Military        6
workplace       5
Other\n         3
religious       1
Airport         1
\nWorkplace     1
Name: location.1, dtype: int64

In [378]:
df['location.1'] = df['location.1'].replace({
    'workplace': 'Workplace',
    'Other\n': 'Other',
    'religious': 'Religious',
    '\nWorkplace': 'Workplace'
})
df['location.1'].value_counts()

Other        53
Workplace    51
School       22
Religious     8
Military      6
Airport       1
Name: location.1, dtype: int64

In [379]:
# Define the age ranges and labels
df['age_of_shooter'] = df['age_of_shooter'].replace({
    '-': 0
})
df['age_of_shooter'] = df['age_of_shooter'].astype(int)
age_ranges = [0, 10, 18, 39, 59, 120]
age_labels = ['Unknown', 'Teenage', 'Early Adulthood', 'Middle Adulthood', 'Old Age']

# Use the cut() function to categorize the age groups
df['age_group'] = pd.cut(df['age_of_shooter'], bins=age_ranges, labels=age_labels)
df['age_group'].value_counts()

Early Adulthood     79
Middle Adulthood    43
Teenage             11
Old Age              6
Unknown              0
Name: age_group, dtype: int64

In [380]:
df.head()

,case,location,date,summary,fatalities,injured,total_victims,location.1,age_of_shooter,prior_signs_mental_health_issues,...,gender,latitude,longitude,type,year,quarter,half,month_name,day_of_week,age_group
0,Nashville religious school shooting,"Nashville, TN",2023-03-27,"Audrey Hale, 28, who was a former student at t...",6,1,6,School,28,Unclear,...,Female (transgender),-,-,Mass,2023,1,1,March,Monday,Early Adulthood
1,Michigan State University shooting,"East Lansing, Michigan",2023-02-13,"Anthony D. McRae, 43, opened fire at Berkey Ha...",3,5,8,School,43,Unclear,...,Male,-,-,Mass,2023,1,1,February,Monday,Middle Adulthood
2,Half Moon Bay spree shooting,"Half Moon Bay, California",2023-01-23,"Chunli Zhao, 67, suspected of carrying out the...",7,1,8,Workplace,67,Unclear,...,Male,-,-,Spree,2023,1,1,January,Monday,Old Age
3,LA dance studio mass shooting,"Monterey Park, California",2023-01-21,"Huu Can Tran, 72, fled the scene in a white va...",11,10,21,Other,72,Yes,...,Male,-,-,Mass,2023,1,1,January,Saturday,Old Age
4,Virginia Walmart shooting,"Chesapeake, Virginia",2022-11-22,"Andre Bing, 31, who worked as a supervisor at ...",6,6,12,Workplace,31,Unclear,...,Male,-,-,Mass,2022,4,2,November,Tuesday,Early Adulthood


In [381]:
df.describe()

,fatalities,age_of_shooter,year,quarter,half
count,141.000000,141.000000,141.000000,141.000000,141.000000
mean,7.808511,33.531915,2010.382979,2.404255,1.460993
std,7.463162,13.726696,10.796600,1.140042,0.500253
min,3.000000,0.000000,1982.000000,1.000000,1.000000
25%,4.000000,22.000000,2005.000000,1.000000,1.000000
50%,6.000000,32.000000,2014.000000,2.000000,1.000000
75%,8.000000,43.000000,2018.000000,3.000000,2.000000
max,58.000000,72.000000,2023.000000,4.000000,2.000000


In [382]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141 entries, 0 to 140
Data columns (total 26 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   case                              141 non-null    object        
 1   location                          141 non-null    object        
 2   date                              141 non-null    datetime64[ns]
 3   summary                           141 non-null    object        
 4   fatalities                        141 non-null    int64         
 5   injured                           141 non-null    object        
 6   total_victims                     141 non-null    object        
 7   location.1                        141 non-null    object        
 8   age_of_shooter                    141 non-null    int64         
 9   prior_signs_mental_health_issues  141 non-null    object        
 10  mental_health_details             141 non-null    

In [383]:
# Create a new column for the decade
df['decade'] = (df['date'].dt.year // 10) * 10

# Convert the decade to a string format
df['decade'] = df['decade'].astype(str) + "'s"

In [384]:
df.head()

,case,location,date,summary,fatalities,injured,total_victims,location.1,age_of_shooter,prior_signs_mental_health_issues,...,latitude,longitude,type,year,quarter,half,month_name,day_of_week,age_group,decade
0,Nashville religious school shooting,"Nashville, TN",2023-03-27,"Audrey Hale, 28, who was a former student at t...",6,1,6,School,28,Unclear,...,-,-,Mass,2023,1,1,March,Monday,Early Adulthood,2020's
1,Michigan State University shooting,"East Lansing, Michigan",2023-02-13,"Anthony D. McRae, 43, opened fire at Berkey Ha...",3,5,8,School,43,Unclear,...,-,-,Mass,2023,1,1,February,Monday,Middle Adulthood,2020's
2,Half Moon Bay spree shooting,"Half Moon Bay, California",2023-01-23,"Chunli Zhao, 67, suspected of carrying out the...",7,1,8,Workplace,67,Unclear,...,-,-,Spree,2023,1,1,January,Monday,Old Age,2020's
3,LA dance studio mass shooting,"Monterey Park, California",2023-01-21,"Huu Can Tran, 72, fled the scene in a white va...",11,10,21,Other,72,Yes,...,-,-,Mass,2023,1,1,January,Saturday,Old Age,2020's
4,Virginia Walmart shooting,"Chesapeake, Virginia",2022-11-22,"Andre Bing, 31, who worked as a supervisor at ...",6,6,12,Workplace,31,Unclear,...,-,-,Mass,2022,4,2,November,Tuesday,Early Adulthood,2020's


Time to save `df` to csv. Data looks clean and informative.

In [385]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141 entries, 0 to 140
Data columns (total 27 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   case                              141 non-null    object        
 1   location                          141 non-null    object        
 2   date                              141 non-null    datetime64[ns]
 3   summary                           141 non-null    object        
 4   fatalities                        141 non-null    int64         
 5   injured                           141 non-null    object        
 6   total_victims                     141 non-null    object        
 7   location.1                        141 non-null    object        
 8   age_of_shooter                    141 non-null    int64         
 9   prior_signs_mental_health_issues  141 non-null    object        
 10  mental_health_details             141 non-null    

## Save Cleaned Dataset and Double Check!

In [386]:
# Saving DataFrame to csv
df.to_csv('shooting_1982-2023_clean.csv', index=False)

In [387]:
df_clean = pd.read_csv("/content/shooting_1982-2023_clean.csv", parse_dates=['date'])

In [388]:
df_clean

,case,location,date,summary,fatalities,injured,total_victims,location.1,age_of_shooter,prior_signs_mental_health_issues,...,latitude,longitude,type,year,quarter,half,month_name,day_of_week,age_group,decade
0,Nashville religious school shooting,"Nashville, TN",2023-03-27,"Audrey Hale, 28, who was a former student at t...",6,1,6,School,28,Unclear,...,-,-,Mass,2023,1,1,March,Monday,Early Adulthood,2020's
1,Michigan State University shooting,"East Lansing, Michigan",2023-02-13,"Anthony D. McRae, 43, opened fire at Berkey Ha...",3,5,8,School,43,Unclear,...,-,-,Mass,2023,1,1,February,Monday,Middle Adulthood,2020's
2,Half Moon Bay spree shooting,"Half Moon Bay, California",2023-01-23,"Chunli Zhao, 67, suspected of carrying out the...",7,1,8,Workplace,67,Unclear,...,-,-,Spree,2023,1,1,January,Monday,Old Age,2020's
3,LA dance studio mass shooting,"Monterey Park, California",2023-01-21,"Huu Can Tran, 72, fled the scene in a white va...",11,10,21,Other,72,Yes,...,-,-,Mass,2023,1,1,January,Saturday,Old Age,2020's
4,Virginia Walmart shooting,"Chesapeake, Virginia",2022-11-22,"Andre Bing, 31, who worked as a supervisor at ...",6,6,12,Workplace,31,Unclear,...,-,-,Mass,2022,4,2,November,Tuesday,Early Adulthood,2020's
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,Shopping centers spree killings,"Palm Bay, Florida",1987-04-23,"Retired librarian William Cruse, 59, was paran...",6,14,20,Other,59,Yes,...,28.0331886,-80.6429695,Spree,1987,2,1,April,Thursday,Middle Adulthood,1980's
137,United States Postal Service shooting,"Edmond, Oklahoma",1986-08-20,"Postal worker Patrick Sherrill, 44, opened fir...",15,6,21,Workplace,44,Unclear,...,35.6672015,-97.42937037,Mass,1986,3,2,August,Wednesday,Middle Adulthood,1980's
138,San Ysidro McDonald's massacre,"San Ysidro, California",1984-07-18,"James Oliver Huberty, 41, opened fire in a McD...",22,19,41,Other,41,Yes,...,32.5520013,-117.0430813,Mass,1984,3,2,July,Wednesday,Middle Adulthood,1980's
139,Dallas nightclub shooting,"Dallas, Texas",1984-06-29,"Abdelkrim Belachheb, 39, opened fire at an ups...",6,1,7,Other,39,Yes,...,32.925166,-96.838676,Mass,1984,2,1,June,Friday,Early Adulthood,1980's


In [389]:
df = df_clean

In [390]:
df['total_victims'] = df['total_victims'].replace({"TK": 10})
df['injured'] = df['injured'].replace({'(TK - "fewer than 10"': 6})
df.head()

,case,location,date,summary,fatalities,injured,total_victims,location.1,age_of_shooter,prior_signs_mental_health_issues,...,latitude,longitude,type,year,quarter,half,month_name,day_of_week,age_group,decade
0,Nashville religious school shooting,"Nashville, TN",2023-03-27,"Audrey Hale, 28, who was a former student at t...",6,1,6,School,28,Unclear,...,-,-,Mass,2023,1,1,March,Monday,Early Adulthood,2020's
1,Michigan State University shooting,"East Lansing, Michigan",2023-02-13,"Anthony D. McRae, 43, opened fire at Berkey Ha...",3,5,8,School,43,Unclear,...,-,-,Mass,2023,1,1,February,Monday,Middle Adulthood,2020's
2,Half Moon Bay spree shooting,"Half Moon Bay, California",2023-01-23,"Chunli Zhao, 67, suspected of carrying out the...",7,1,8,Workplace,67,Unclear,...,-,-,Spree,2023,1,1,January,Monday,Old Age,2020's
3,LA dance studio mass shooting,"Monterey Park, California",2023-01-21,"Huu Can Tran, 72, fled the scene in a white va...",11,10,21,Other,72,Yes,...,-,-,Mass,2023,1,1,January,Saturday,Old Age,2020's
4,Virginia Walmart shooting,"Chesapeake, Virginia",2022-11-22,"Andre Bing, 31, who worked as a supervisor at ...",6,6,12,Workplace,31,Unclear,...,-,-,Mass,2022,4,2,November,Tuesday,Early Adulthood,2020's


In [391]:
# split the "summary" column by ","
summary_split = df['summary'].str.split(',', expand=True)

# rename the new columns to 'name', 'current_age', and 'description'
summary_split.columns = ['name', 'current_age', 'description', 'col3', 'col4', 'col5', 'col6', 'col7', 'col8', 'col9']

# drop the extra columns
summary_split = summary_split[['name', 'current_age', 'description']]

# concatenate the new columns with the original DataFrame
df = pd.concat([df, summary_split], axis=1)

# display the updated DataFrame
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141 entries, 0 to 140
Data columns (total 30 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   case                              141 non-null    object        
 1   location                          141 non-null    object        
 2   date                              141 non-null    datetime64[ns]
 3   summary                           141 non-null    object        
 4   fatalities                        141 non-null    int64         
 5   injured                           141 non-null    object        
 6   total_victims                     141 non-null    object        
 7   location.1                        141 non-null    object        
 8   age_of_shooter                    141 non-null    int64         
 9   prior_signs_mental_health_issues  141 non-null    object        
 10  mental_health_details             141 non-null    

In [392]:
df.columns

Index(['case', 'location', 'date', 'summary', 'fatalities', 'injured',
       'total_victims', 'location.1', 'age_of_shooter',
       'prior_signs_mental_health_issues', 'mental_health_details',
       'weapons_obtained_legally', 'where_obtained', 'weapon_type',
       'weapon_details', 'race', 'gender', 'latitude', 'longitude', 'type',
       'year', 'quarter', 'half', 'month_name', 'day_of_week', 'age_group',
       'decade', 'name', 'current_age', 'description'],
      dtype='object')

In [393]:
#df = df.iloc[:, :-3]

In [394]:
df.head()

,case,location,date,summary,fatalities,injured,total_victims,location.1,age_of_shooter,prior_signs_mental_health_issues,...,year,quarter,half,month_name,day_of_week,age_group,decade,name,current_age,description
0,Nashville religious school shooting,"Nashville, TN",2023-03-27,"Audrey Hale, 28, who was a former student at t...",6,1,6,School,28,Unclear,...,2023,1,1,March,Monday,Early Adulthood,2020's,Audrey Hale,28,who was a former student at the private Coven...
1,Michigan State University shooting,"East Lansing, Michigan",2023-02-13,"Anthony D. McRae, 43, opened fire at Berkey Ha...",3,5,8,School,43,Unclear,...,2023,1,1,February,Monday,Middle Adulthood,2020's,Anthony D. McRae,43,opened fire at Berkey Hall and the MSU union
2,Half Moon Bay spree shooting,"Half Moon Bay, California",2023-01-23,"Chunli Zhao, 67, suspected of carrying out the...",7,1,8,Workplace,67,Unclear,...,2023,1,1,January,Monday,Old Age,2020's,Chunli Zhao,67,suspected of carrying out the attacks at a mu...
3,LA dance studio mass shooting,"Monterey Park, California",2023-01-21,"Huu Can Tran, 72, fled the scene in a white va...",11,10,21,Other,72,Yes,...,2023,1,1,January,Saturday,Old Age,2020's,Huu Can Tran,72,fled the scene in a white van and later shot ...
4,Virginia Walmart shooting,"Chesapeake, Virginia",2022-11-22,"Andre Bing, 31, who worked as a supervisor at ...",6,6,12,Workplace,31,Unclear,...,2022,4,2,November,Tuesday,Early Adulthood,2020's,Andre Bing,31,who worked as a supervisor at the store


In [395]:
#@title Fill empty longitude/latitude with Google Maps

!pip install googlemaps
import googlemaps
from geopy.exc import GeocoderTimedOut
#@markdown Please enteter your Google Maps API Key kere
# set up Google Maps client with API key
KEY = "YOURAPIHERE" #@param 
gmaps = googlemaps.Client(key=KEY) 

# define function to geocode location and return latitude and longitude
def geocode_location(location):
    try:
        geocode_result = gmaps.geocode(location)
        lat = geocode_result[0]['geometry']['location']['lat']
        lng = geocode_result[0]['geometry']['location']['lng']
        return lat, lng
    except (IndexError, KeyError, GeocoderTimedOut):
        return None, None

# apply the function to update missing values in the DataFrame
geocoded = df.loc[(df['latitude'] == '-') | (df['longitude'] == '-'), 'location'].apply(geocode_location).tolist()
df.loc[(df['latitude'] == '-') | (df['longitude'] == '-'), ['latitude', 'longitude']] = geocoded


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [396]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141 entries, 0 to 140
Data columns (total 30 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   case                              141 non-null    object        
 1   location                          141 non-null    object        
 2   date                              141 non-null    datetime64[ns]
 3   summary                           141 non-null    object        
 4   fatalities                        141 non-null    int64         
 5   injured                           141 non-null    object        
 6   total_victims                     141 non-null    object        
 7   location.1                        141 non-null    object        
 8   age_of_shooter                    141 non-null    int64         
 9   prior_signs_mental_health_issues  141 non-null    object        
 10  mental_health_details             141 non-null    

In [397]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141 entries, 0 to 140
Data columns (total 30 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   case                              141 non-null    object        
 1   location                          141 non-null    object        
 2   date                              141 non-null    datetime64[ns]
 3   summary                           141 non-null    object        
 4   fatalities                        141 non-null    int64         
 5   injured                           141 non-null    object        
 6   total_victims                     141 non-null    object        
 7   location.1                        141 non-null    object        
 8   age_of_shooter                    141 non-null    int64         
 9   prior_signs_mental_health_issues  141 non-null    object        
 10  mental_health_details             141 non-null    

## Save cleaned Data

In [398]:
# Saving DataFrame to csv
df.to_csv('shooting_1982-2023_cleaned.csv', index=False)

In [399]:
df = pd.read_csv("shooting_1982-2023_cleaned.csv")
len(df)

141

# Part 2: Visualize